In [1]:
import sys
import tqdm
import time
import sklearn
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from load_dataset import load, generate_random_dataset
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from scipy import stats
from scipy.stats import rankdata
from sklearn import metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from operator import itemgetter
from torch.autograd import grad
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

**Load Dataset**

In [3]:
dataset = 'german'
X_train, X_test, y_train, y_test = load(dataset)

In [4]:
duplicates = 1
make_duplicates = lambda x, d: pd.concat([x]*d, axis=0).reset_index(drop=True)
X_train = make_duplicates(X_train, duplicates)
X_test = make_duplicates(X_test, duplicates)
y_train = make_duplicates(y_train, duplicates)
y_test = make_duplicates(y_test, duplicates)

**Parametric Model**

In [5]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Loss function** (Log loss for logistic regression)

In [6]:
# clf = NeuralNetwork(input_size=X_train.shape[-1])
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])
num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression) or isinstance(clf, NeuralNetwork):
    loss_func = logistic_loss_torch
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch

**Influence of points computed using ground truth**

In [7]:
def ground_truth_influence(X_train, y_train, X_test, X_test_orig, y_test):
    clf.fit(X_train, y_train, verbose=True)
    y_pred = clf.predict_proba(X_test)
    spd_0 = computeFairness(y_pred, X_test_orig, y_test, 0)

    delta_spd = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba(X_test)
        delta_spd_i = computeFairness(y_pred, X_test_orig, y_test, 0) - spd_0
        delta_spd.append(delta_spd_i)

    return delta_spd

**Compute Accuracy** 

In [8]:
def computeAccuracy(y_true, y_pred):
    return np.sum((y_pred>0.5) == y_true)/len(y_pred)

**First-order derivative of loss function at z with respect to model parameters**

In [9]:
def del_L_del_theta_i(model, x, y_true, retain_graph=False):
    loss = loss_func(model, x, y_true)
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(loss, w, create_graph=True, retain_graph=retain_graph)

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [10]:
def del_f_del_theta_i(model, x, retain_graph=False):
    w = [ p for p in model.parameters() if p.requires_grad ]
    return grad(model(torch.FloatTensor(x)), w, retain_graph=retain_graph)

**Stochastic estimation of Hessian vector product (involving del fairness): $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [11]:
def hvp(y, w, v):
    ''' Multiply the Hessians of y and w by v.'''
    # First backprop
    first_grads = grad(y, w, retain_graph=True, create_graph=True)

    # Elementwise products
    elemwise_products = 0
    for grad_elem, v_elem in zip(convert_grad_to_tensor(first_grads), v):
        elemwise_products += torch.sum(grad_elem * v_elem)

    # Second backprop
    return_grads = grad(elemwise_products, w, create_graph=True)

    return return_grads

In [12]:
def hessian_one_point(model, x, y):
    x, y = torch.FloatTensor(x), torch.FloatTensor([y])
    loss = loss_func(model, x, y)
    params = [ p for p in model.parameters() if p.requires_grad ]
    first_grads = convert_grad_to_tensor(grad(loss, params, retain_graph=True, create_graph=True))
    hv = np.zeros((len(first_grads), len(first_grads)))
    for i in range(len(first_grads)):
        hv[i, :] = convert_grad_to_ndarray(grad(first_grads[i], params, create_graph=True)).ravel()
    return hv

In [13]:
# Compute multiplication of inverse hessian matrix and vector v
def s_test(model, xs, ys, v, hinv=None, damp=0.01, scale=25.0, r=-1, batch_size=-1, recursive=False, verbose=False):
    ''' Arguments:
        xs: list of data points
        ys: list of true labels corresponding to data points in xs
        damp: dampening factor
        scale: scaling factor
        r: number of iterations aka recursion depth
            should be enough so that the value stabilises.
        batch_size: number of instances in each batch in recursive approximation
        recursive: determine whether to recursively approximate hinv_v'''
    xs, ys = torch.FloatTensor(xs.copy()), torch.FloatTensor(ys.copy())
    n = len(xs)
    if recursive:
        hinv_v = copy.deepcopy(v)
        if verbose:
            print('Computing s_test...')
            tbar = tqdm.tqdm(total=r)
        if (batch_size == -1):  # default
            batch_size = 10
        if (r == -1):
            r = n // batch_size + 1
        sample = np.random.choice(range(n), r*batch_size, replace=True)
        for i in range(r):
            sample_idx = sample[i*batch_size:(i+1)*batch_size]
            x, y = xs[sample_idx], ys[sample_idx]
            loss = loss_func(model, x, y)
            params = [ p for p in model.parameters() if p.requires_grad ]
            hv = convert_grad_to_ndarray(hvp(loss, params, torch.FloatTensor(hinv_v)))
            # Recursively caclulate h_estimate
            hinv_v = v + (1 - damp) * hinv_v - hv / scale
            if verbose:
                tbar.update(1)
    else:
        if hinv is None:
            hinv = np.linalg.pinv(np.sum(hessian_all_points, axis=0))
        scale = 1.0
        hinv_v = np.matmul(hinv, v)

    return hinv_v / scale

**Metrics: Initial state**

In [14]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.09508493496105075
Initial TPR parity:  -0.07770497195525183
Initial predictive parity:  -0.10142464519673511
Initial loss:  0.5078865702279716
Initial accuracy:  0.755


In [15]:
hessian_all_points = []
tbar = tqdm.tqdm(total=len(X_train))
total_time = 0
for i in range(len(X_train)):
    t0 = time.time()
    hessian_all_points.append(hessian_one_point(clf, X_train[i], y_train[i])/len(X_train))
    total_time += time.time()-t0
    tbar.update(1)

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 795/800 [00:06<00:00, 121.45it/s]

In [16]:
hessian_all_points = np.array(hessian_all_points)

In [17]:
total_time

6.600985288619995

**Pre-compute: (1) Hessian (2) del_L_del_theta for each training data point**

In [18]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    gradient = convert_grad_to_ndarray(del_L_del_theta_i(clf, X_train[i], int(y_train[i])))
    while np.sum(np.isnan(gradient))>0:
        gradient = convert_grad_to_ndarray(del_L_del_theta_i(clf, X_train[i], int(y_train[i])))
    del_L_del_theta.append(gradient)
del_L_del_theta = np.array(del_L_del_theta)

*Select delta fairness function depending on selected metric*

In [19]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(clf, X_test_orig, X_test, dataset)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(clf, X_test_orig, X_test, y_test, dataset)

In [20]:
hinv = np.linalg.pinv(np.sum(hessian_all_points, axis=0))
hinv_v = s_test(clf, X_train, y_train, v1, hinv=hinv, verbose=False)

**First-order influence computation**

In [21]:
def first_order_influence(del_L_del_theta, hinv_v, n):
    infs = []
    for i in range(n):
        inf = -np.dot(del_L_del_theta[i].transpose(), hinv_v)
        inf *= -1/n
        infs.append(inf)
    return infs

In [22]:
def first_order_group_influence(U, del_L_del_theta):
    infs = []
    u = len(U)
    n = len(X_train)
    for i in range(u):
        idx = U[i]
        inf = -np.dot(del_L_del_theta[idx].transpose(), hinv)
        inf *= -1/n
        infs.append(inf)
    return np.sum(infs, axis=0)

**Second-order influence computation for a group of points in subset U**

In [23]:
def second_order_influence(model, X_train, y_train, U, del_L_del_theta, r=-1, verbose=False):
    u = len(U)
    s = len(X_train)
    p = u/s
    c1 = (1 - 2*p)/(s * (1-p)**2)
    c2 = 1/((s * (1-p))**2)
    num_params = len(del_L_del_theta[0])
    del_L_del_theta_sum = np.sum([del_L_del_theta[i] for i in U], axis=0)
    hinv_del_L_del_theta= s_test(model, X_train, y_train, del_L_del_theta_sum, hinv=hinv)
    hessian_U_hinv_del_L_del_theta = np.zeros((num_params,))
    for i in range(u):
        idx = U[i]
        x, y = torch.FloatTensor(X_train[idx]), torch.FloatTensor([y_train[idx]])
        loss = loss_func(model, x, y)
        params = [ p for p in model.parameters() if p.requires_grad ]
        hessian_U_hinv_del_L_del_theta += convert_grad_to_ndarray(hvp(loss, params, torch.FloatTensor(hinv_del_L_del_theta)))

    term1 = c1 * hinv_del_L_del_theta
    term2 = c2 * s_test(model, X_train, y_train, hessian_U_hinv_del_L_del_theta, hinv=hinv)
    sum_term = term1 + term2
    return sum_term

In [24]:
def second_order_group_influence(U, del_L_del_theta):
    u = len(U)
    s = len(X_train)
    p = u/s
    c1 = (1 - 2*p)/(s * (1-p)**2)
    c2 = 1/((s * (1-p))**2)
    num_params = len(del_L_del_theta[0])
    del_L_del_theta_sum = np.sum(del_L_del_theta[U, :], axis=0)
    hinv_del_L_del_theta= np.matmul(hinv, del_L_del_theta_sum)
    hessian_U_hinv_del_L_del_theta = np.sum(np.matmul(hessian_all_points[U, :], hinv_del_L_del_theta), axis=0)
    term1 = c1 * hinv_del_L_del_theta
    term2 = c2 * np.matmul(hinv, hessian_U_hinv_del_L_del_theta)
    sum_term = (term1 + term2*len(X_train))
    return sum_term

**First-order influence of each training data point**

In [25]:
infs_1 = first_order_influence(del_L_del_theta, hinv_v, len(X_train))

**Fairness: Ground-truth subset influence vs. computed subset influences: Coherent subset** 

(by coherent, we mean group of data points that share some properties)

***NOTE:*** The retraining of the clf would cause the change in model parameters and thus lead to the change of gradients, so in this part, we first acquire all the first- and second-order influence functions together based on the original model. After all the influence functions are calculated, we retrain the model corresponding to different removed coherent subset of data and get the ground truth.

In [26]:
time_gt = []
time_first = []
time_second = []
rep = 1

In [27]:
alpha_f_lower = (-0.01) * (spd_0)
alpha_f_upper = -spd_0
del_f_threshold = (0.1) * spd_0
support = 0.05 # Do not consider extremely small patterns
support_small = 0.3 # For small patterns, 2nd-order estimation is quite accurate
del_f_threshold_small = (-0.1) * (spd_0)
print("alpha_f_lower:", alpha_f_lower)
print("alpha_f_upper:", alpha_f_upper)
print("del_f_threshold:", del_f_threshold)
print("support_small:", support_small)
print("del_f_threshold_small:", del_f_threshold_small)

alpha_f_lower: 0.0009508493496105075
alpha_f_upper: 0.09508493496105075
del_f_threshold: -0.009508493496105075
support_small: 0.3
del_f_threshold_small: 0.009508493496105075


In [28]:
for _ in range(rep):
#     Get the original model
#     clf = NeuralNetwork(input_size=X_train.shape[-1])
    clf = LogisticRegression(input_size=X_train.shape[-1])
#     clf = SVM()
    clf.fit(X_train, y_train)

    attributes = []
    attributeValues = []
    first_order_influences = []
    second_order_influences = []
    fractionRows = []

    v1_orig = v1
    for col in X_train_orig.columns:
        if dataset == 'german':
            if "purpose" in col or "housing" in col: #dummy variables purpose=0 doesn't make sense
                vals = [1]
            else:
                vals = X_train_orig[col].unique()
        elif dataset == 'adult':
            vals = X_train_orig[col].unique()
        elif dataset == 'compas':
            vals = X_train_orig[col].unique()
        elif dataset == 'sqf':
            vals = X_train_orig[col].unique()
        elif dataset == 'random':
            vals = X_train_orig[col].unique()
        else:
            raise NotImplementedError
        for val in vals:
#             print(col, val, sep=": ")
            idx = X_train_orig[X_train_orig[col] == val].index  
            if (len(idx)/len(X_train) > support):  
                X = np.delete(X_train, idx, 0)
                y = y_train.drop(index=idx, inplace=False)
                if len(y.unique()) > 1:
                    idx = X_train_orig[X_train_orig[col] == val].index 

                    # First-order subset influence
                    t0 = time.time()
                    del_f_1 = 0            

                    # Second-order subset influence
                    t0 = time.time()
                    params_f_2 = second_order_group_influence(idx, del_L_del_theta)
                    del_f_2 = np.dot(v1.transpose(), params_f_2)
                    time_second.append(time.time()-t0)

                    attributes.append(col)
                    attributeValues.append(val)
                    first_order_influences.append(del_f_1)
                    second_order_influences.append(del_f_2)
                    fractionRows.append(len(idx)/len(X_train)*100)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:19<00:00, 121.45it/s]

In [29]:
expl = [attributes, attributeValues, first_order_influences, second_order_influences, fractionRows]
expl = (np.array(expl).T).tolist()

explanations = pd.DataFrame(expl, columns=["attributes", "attributeValues", "first_order_influences", "second_order_influences", "fractionRows"])
explanations['second_order_influences'] = explanations['second_order_influences'].astype(float)
explanations['first_order_influences'] = explanations['first_order_influences'].astype(float)
explanations['fractionRows'] = explanations['fractionRows'].astype(float)

In [30]:
candidates = copy.deepcopy(explanations)
candidates.loc[:, 'score'] = candidates.loc[:, 'second_order_influences']*100/candidates.loc[:, 'fractionRows']
# display(candidates)

In [31]:
len(candidates)

62

In [32]:
def get_subset(explanation):
    subset = X_train_orig.copy()
    for predicate in explanation:
        attr = predicate.split("=")[0].strip(' ')
        val = int(predicate.split("=")[1].strip(' '))
        subset = subset[subset[attr]==val]
    return subset.index

In [34]:
%%time
candidates_all = []
total_rows = len(X_train_orig)

# Generating 1-candidates
candidates_1 = []
for i in range(len(candidates)):
    candidate = []
    candidate_i = candidates.iloc[i]
    if ((candidate_i["fractionRows"] >= support_small) or
       ((candidate_i["fractionRows"] >= support) & (candidate_i["second_order_influences"] > del_f_threshold))
       ):
        attr_i = candidate_i["attributes"]
        val_i = int(float(candidate_i["attributeValues"]))
        idx = X_train_orig[X_train_orig[attr_i] == val_i].index
        predicates = [attr_i + '=' + str(val_i)]
        candidate = [predicates, candidate_i["fractionRows"],
                     candidate_i["score"], candidate_i["second_order_influences"], idx]
        candidates_1.append(candidate)

print("Generated: ", len(candidates_1), " 1-candidates")
candidates_1.sort()
# display(candidates_1)

for i in range(len(candidates_1)):
    if (float(candidates_1[i][2]) >= support): # if score > top-k, keep in candidates, not otherwise
        candidates_all.insert(len(candidates_all), candidates_1[i])

# Generating 2-candidates
candidates_2 = []
for i in range(len(candidates_1)):
    predicate_i = candidates_1[i][0][0]
    attr_i = predicate_i.split("=")[0]
    val_i = int(float(predicate_i.split("=")[1]))
    sup_i = candidates_1[i][1]
    idx_i = candidates_1[i][-1]
    for j in range(i):
        predicate_j = candidates_1[j][0][0]
        attr_j = predicate_j.split("=")[0]
        val_j = int(float(predicate_j.split("=")[1]))
        sup_j = candidates_1[j][1]
        idx_j = candidates_1[j][-1]
        if (attr_i != attr_j):
            idx = idx_i.intersection(idx_j)
            fractionRows = len(idx)/total_rows * 100
            isCompact = True
            if (fractionRows == min(sup_i, sup_j)): # pattern is not compact if intersection equals one of its parents
                isCompact = False
            if (fractionRows/100 >= support):
                params_f_2 = second_order_group_influence(idx, del_L_del_theta)
                del_f_2 = np.dot(v1.transpose(), params_f_2)
                score = del_f_2 * 100/fractionRows
                if ((fractionRows/100 >= support_small) or
                    ((score > candidates_1[i][2]) & (score > candidates_1[j][2]))):
                        predicates = [attr_i + '=' + str(val_i), attr_j + '=' + str(val_j)]
                        candidate = [sorted(predicates, key=itemgetter(0)), len(idx)*100/total_rows,
                                    score, del_f_2, idx]
                        candidates_2.append(candidate)
#                         print(candidate)
                        if (isCompact):
                            candidates_all.append(candidate)
print("Generated: ", len(candidates_2), " 2-candidates")

# Recursively generating the rest
candidates_L_1 = copy.deepcopy(candidates_2)
set_L_1 = set()
iteration = 2
while((len(candidates_L_1) > 0) & (iteration < 4)):
    print("Generated: ", iteration)    
    candidates_L = []
    for i in range(len(candidates_L_1)):
        candidate_i = set(candidates_L_1[i][0])
        sup_i = candidates_L_1[i][1]
        idx_i = candidates_L_1[i][-1]
        for j in range(i):
            candidate_j = set(candidates_L_1[j][0])
            sup_j = candidates_L_1[j][1]
            idx_j = candidates_L_1[j][-1]
            merged_candidate = sorted(candidate_i.union(candidate_j), key=itemgetter(0))
            if json.dumps(merged_candidate) in set_L_1:
                continue
            if (len(merged_candidate) == iteration + 1):
                intersect_candidates = candidate_i.intersection(candidate_j)
                setminus_i = list(candidate_i - intersect_candidates)[0].split("=")
                setminus_j = list(candidate_j - intersect_candidates)[0].split("=")
                attr_i = setminus_i[0]
                val_i = int(setminus_i[1])
                attr_j = setminus_j[0]
                val_j = int(setminus_j[1])
                if (attr_i != attr_j):
                    # merge to get L list
                    idx = idx_i.intersection(idx_j)
                    fractionRows = len(idx)/len(X_train) * 100
                    isCompact = True
                    if (fractionRows == min(sup_i, sup_j)): # pattern is not compact if intersection equals one of its parents
                        isCompact = False
                    if (fractionRows/100 >= support):
                        X = np.delete(X_train, idx, 0)
                        y = y_train.drop(index=idx, inplace=False)

                        params_f_2 = second_order_group_influence(idx, del_L_del_theta)
                        del_f_2 = np.dot(v1.transpose(), params_f_2)

                        score = del_f_2 * 100/fractionRows
                        if (((score > candidates_L_1[i][2]) & (score > candidates_L_1[j][2])) or 
                           (fractionRows >= support_small)):
                            candidate = [merged_candidate, fractionRows,
                                         del_f_2*len(X_train)/len(idx), del_f_2, idx]
                            candidates_L.append(candidate)
                            set_L_1.add(json.dumps(merged_candidate))
                            if (isCompact):
                                candidates_all.insert(len(candidates_all), candidate)
    set_L_1 = set()
    print("Generated:", len(candidates_L), " ", str(iteration+1), "-candidates")
    candidates_L_1 = copy.deepcopy(candidates_L)
    candidates_L_1.sort()
    iteration += 1

Generated:  62  1-candidates
Generated:  341  2-candidates
Generated:  2
Generated: 2418   3 -candidates
Generated:  3
Generated: 10809   4 -candidates
CPU times: user 17.1 s, sys: 107 ms, total: 17.2 s
Wall time: 17.2 s


In [35]:
candidates_support_3_compact = copy.deepcopy(candidates_all)
print(len(candidates_support_3_compact))
candidates_df_3_compact = pd.DataFrame(candidates_support_3_compact, columns=["predicates","support","score","2nd-inf",'idx'])
candidates_df_3_compact = candidates_df_3_compact.sort_values(by=['score'], ascending=False)
print(len(candidates_df_3_compact))
# display(candidates_df_3_compact)
# display(candidates_df_3_compact[candidates_df_3_compact["support"] < 20].sort_values(by=['2nd-inf'], ascending=False).head(5))

13409
13409


**Containment-based filtering**

In [36]:
class Topk:
    """
        top explanations: explanation -> (minhash, set_index, score)
    """

    def __init__(self, method='containment', threshold=0.75, k=5):
        self.method = method
        if method == 'lshensemble':
            raise NotImplementedError
        elif method == 'lsh':
            raise NotImplementedError

        self.top_explanations = dict()
        self.k = k
        self.threshold = threshold
        self.min_score = -100
        self.min_score_explanation = None
        self.containment_hist = []

    def _update_min(self, new_explanation, new_score):
        if len(self.top_explanations) > 0:
            for explanation, t in self.top_explanations.items():
                if t[1] < new_score:
                    new_score = t[1]
                    new_explanation = explanation
        self.min_score = new_score
        self.min_score_explanation = new_explanation

    def _containment(self, x, q):
        c = len(x & q) / len(q)
        self.containment_hist.append(c)
        return c

    def update(self, explanation, score):
        if (len(self.top_explanations) < self.k) or (score > self.min_score):
            s = get_subset(explanation)
            explanation = json.dumps(explanation)

            if self.method == 'lshensemble':
                raise NotImplementedError
            elif self.method == 'lsh':
                raise NotImplementedError
            elif self.method == 'containment':
                q_result = set()
                for k, v in self.top_explanations.items():
                    if self._containment(v[0], s) > self.threshold:
                        q_result.add(k)

            if len(q_result) == 0:
                if len(self.top_explanations) <= self.k - 1:
                    self._update_min(explanation, score)
                    self.top_explanations[explanation] = (s, score)
                    return 0
        return -1

In [37]:
support_threshold = 0.2
containment_df = candidates_df_3_compact[candidates_df_3_compact['support'] < support_threshold*100].sort_values(by=['score'],
                                                                                              ascending=False).copy()

topk = Topk(method='containment', threshold=0.2, k=5)
for row_idx in range(len(containment_df)):
    row = containment_df.iloc[row_idx]
    explanation, score = row[0], row[2]
    topk.update(explanation, score)
    if len(topk.top_explanations) == topk.k:
        break

In [38]:
explanations = list(topk.top_explanations.keys())
idxs = [v[1] for v in topk.top_explanations.values()]
supports = list()
scores = list()
gt_scores = list()
infs = list()
gts = list()
new_accs = list()
for e in explanations:
    idx = get_subset(json.loads(e))
    X = np.delete(X_train, idx, 0)
    y = y_train.drop(index=idx, inplace=False)
    clf.fit(np.array(X), np.array(y))
    y_pred = clf.predict_proba(np.array(X_test))
    new_acc = computeAccuracy(y_test, y_pred)
    inf_gt = computeFairness(y_pred, X_test_orig, y_test, 0, dataset) - spd_0
    
    condition = candidates_df_3_compact.predicates.apply(lambda x: x==json.loads(e))
    supports.append(float(candidates_df_3_compact[condition]['support']))
    scores.append(float(candidates_df_3_compact[condition]['score']))
    infs.append(float(candidates_df_3_compact[condition]['2nd-inf']))
    gts.append(inf_gt/(-spd_0))
    gt_scores.append(inf_gt*100/float(candidates_df_3_compact[condition]['support']))
    new_accs.append(new_acc)


expl = [explanations, supports, scores, gt_scores, infs, gts, new_accs]
expl = (np.array(expl).T).tolist()

explanations = pd.DataFrame(expl, columns=["explanations", "support", "score", "gt-score", "2nd-inf(%)", "gt-inf(%)", "new-acc"])
explanations['score'] = explanations['score'].astype(float)
explanations['gt-score'] = explanations['gt-score'].astype(float)
explanations['support'] = explanations['support'].astype(float)
explanations['2nd-inf(%)'] = explanations['2nd-inf(%)'].astype(float)/(-spd_0)
explanations['gt-inf(%)'] = explanations['gt-inf(%)'].astype(float)
explanations['new-acc'] = explanations['new-acc'].astype(float)

pd.set_option('max_colwidth', 100)
explanations.sort_values(by=['score'], ascending=False)


,explanations,support,score,gt-score,2nd-inf(%),gt-inf(%),new-acc
0,"[""age=1"", ""gender=0""]",5.000,0.985302,1.075691,0.518117,0.565648,0.745
1,"[""age=1"", ""credit_hist=0"", ""gender=1""]",6.250,0.621814,0.657572,0.408723,0.432227,0.755
2,"[""credit_amt=0"", ""install_rate=4"", ""install_plans=0"", ""status=0""]",6.500,0.419609,0.410323,0.286844,0.280497,0.755
3,"[""duration=1"", ""employment=4"", ""num_liable=1"", ""residence=4""]",5.375,0.405261,0.400088,0.229088,0.226164,0.740
4,"[""credit_hist=2"", ""gender=0"", ""housing_A152=1"", ""savings=0""]",8.000,0.393449,0.382634,0.331030,0.321931,0.760
